In [1]:
import pandas as pd
import csv
import random


In [ ]:
def load_translation_table(file_path):
    L2S={}
    S2L={} 
    with open(file_path, 'r') as csvfile:
        r = csv.reader(csvfile, delimiter=',', quotechar='"',)
        next(r)
        for row in r:
            if row[0] in L2S.keys(): 
                print("Duplicate id value detected:" + str(row[0]))
            else:
                L2S[row[0]]=row[1]
            if row[1] in S2L.keys():
                print("Duplicate syn value detected:" + str(row[1]))
            else:
                S2L[row[1]]=row[0]
                
    return L2S, S2L


def save_translation_table(file_path,L2S):
    with open(file_path,'w') as f:
        w = csv.writer(f, delimiter=',', quotechar='"',quoting=csv.QUOTE_MINIMAL) 
        w.writerow(['id','syn'])
        w.writerows(L2S.items())




These functions below are meant to manage the available population of SYNs
If you only need to anonymise one file, a straight process to identify unique keys and associate to a series is fine
But if you are going to apply to multiple separate tables, you need to manage the extra ids that may appear down the line

In [ ]:
def create_syn_range(start,end):
    l = list(range(start,end+1))
    l = [str(i) for i in l]
    return l

def load_available_syns(file_path):
    l=[]
    with open(file_path , 'r') as f:
        f.readline()
        for line in f:
            line = line.strip() #or some other preprocessing
            l.append(line) #storing everything in memory!
    return l
def save_available_syns(file_path, list_available):
    with open(file_path , 'w') as f:
        f.write("syn\n")
        for item in list_available:
            f.write("%s\n" % item)

def check_available_syns(list_available, S2L, L2S, cleanup_used= False):
    error_count=0
    for n in S2L.keys():
        if n in list_available:          
            error_count += 1
            loc= list_available.index(n)
            print(str(n)+" was found in the list_available list" + str(loc))
            if cleanup_used==True:
                list_available.pop(loc)
                print ("Removed from list in memory")
    if error_count>0:
        print ("Errors found checking S2L:" + str(error_count))
    else:
        print("No errors found checking S2L")
    error_count=0
    for n in L2S.values():
        if n in list_available:          
            error_count += 1
            loc= list_available.index(n)
            print(str(n)+" was found in the list_available list" + str(loc))
            if cleanup_used==True:
                list_available.pop(loc)
                print ("Removed from list in memory")
    if error_count>0:
        print ("Errors found checking L2S:" + str(error_count))
    else:
        print("No errors found checking L2S")

def expand_available_syns(list_available,  S2L, start, end):
    new_range= list(range(start,end+1))
    for n in new_range:
        if  str(n) not in S2L.keys():
            list_available.append(str(n))
    return list_available

def assign_syn_to_live(live_value,list_available,L2S,S2L):
    if live_value in L2S.keys():
        return L2S[live_value]
    else:
        if len(list_available)>0:
            new_syn=list_available.pop(0)
            L2S[live_value]=new_syn
            S2L[new_syn]=live_value
        else:
            print("Ran out of available ids")
            new_syn=""
        return new_syn


In [ ]:

def anonymise(df, column_name, list_available, L2S, S2L ):
    for i, r in df.iterrows():
        live_value= r[column_name]
        syn_value = assign_syn_to_live(live_value,list_available,L2S,S2L)
        df.at[i, column_name]= syn_value
        
def de_anonymise(df,column_name,list_available,S2L):
    for i, r in df.iterrows():
        syn_value= r[column_name]
        live_value= S2L[syn_value]
        df.at[i,column_name]= live_value 

        
        
        

In [3]:
def anonymise_mass(df,column_name, L2S, S2L):
    trans = df[column_name].drop_duplicates().to_frame()
    trans.sort_values(column_name, inplace=True)
    trans.reset_index(drop=True, inplace=True)
    #This would be how to do it directly without any persistent translation table
    #trans.insert(1, 'New_ID', range(1, len(trans)+1))
    #trans_dict= pd.Series(trans.New_ID.values, index= trans.ID).to_dict()
    for i, r in trans.iterrows():
        live_value= r[column_name]
        syn_value = assign_syn_to_live(live_value,list_available,L2S,S2L)
 
    return L2S

In [5]:
trans_dict= create_translation(df,"ID")
trans_dict[800002]

3